In [74]:
import nltk
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
import os
from collections import defaultdict
import copy
from functools import reduce
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aryan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aryan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aryan\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [75]:
docs = []
docnames = []

for filename in os.listdir('data'):
    try:
        lines = []
        file = open('data/'+filename, 'r')
        for i in file:
            lines.append(i.replace('\n', ''))
        docnames.append(filename)
        docs.append(lines)
    except:
        print('File not read:', filename)

File not read: hilbilly.wri
File not read: howlong.hum
File not read: oxymoron.txt
File not read: steroid.txt
File not read: various.txt


In [76]:
len(docnames)

1128

## A: Preprocessing

In [77]:
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [107]:
def preprocess(doc):
    procd = []
    for line in doc:
        line = line.lower()
        line = word_tokenize(line)
        tokens = []
        for word in line:
            if(word not in stop_words and word.isalnum()):
                procd.append(stemmer.stem(word))
                #procd.append(lemmatizer.lemmatize(word))
                #procd.append(word)
    return procd

In [108]:
prodocs = []

for doc in docs:
    procd = preprocess(doc)
    prodocs.append(procd)

## B: Unigram Inverted Index

In [109]:
index = defaultdict(list)

for i, doc in enumerate(prodocs):
    for j, word in enumerate(doc):
        if word in index:
            if i not in index[word][1]:
                index[word][1].append(i)
            index[word][0] = len(index[word][1])
        else:
            index[word] = [1]
            index[word].append([i])

In [110]:
index['rainbow']

[24,
 [30,
  128,
  200,
  240,
  252,
  295,
  406,
  412,
  450,
  509,
  517,
  518,
  600,
  669,
  720,
  751,
  784,
  812,
  814,
  847,
  912,
  921,
  1031,
  1070]]

## C: Queries Handling

In [111]:
def ander(l1, l2):
    if(len(l2) < len(l1)):
        l1, l2 = l2, l1
    ret = []
    comparisons = 0
    i = j = 0
    
    while(i<len(l1) and j<len(l2)):
        if(l1[i] == l2[j]):
            ret.append(l1[i])
            i += 1
            j += 1
        elif(l1[i] < l2[j]):
            i += 1
        else:
            j += 1
        comparisons += 1
    
    return comparisons, ret

def orer(l1, l2):
    ret = []
    comparisons = 0
    i = j = 0
    
    while(i<len(l1) and j<len(l2)):
        if(l1[i] == l2[j]):
            ret.append(l1[i])
            i += 1
            j += 1
        elif(l1[i] < l2[j]):
            ret.append(l1[i])
            i += 1
        else:
            ret.append(l2[j])
            j += 1
        comparisons += 1
    
    while(i<len(l1)):
        ret.append(l1[i])
        i += 1
    while(j<len(l2)):
        ret.append(l2[j])
        j += 1
    
    return comparisons, ret

def noter(l1):
    idlist = list(range(len(prodocs)))
    for docid in l1:
        idlist.remove(docid)
    return idlist

In [124]:
def handle(l1, l2, op):
    if(op == 'and'):
        return ander(l1, l2)
    elif(op == 'or'):
        return orer(l1, l2)
    elif(op == 'or not'):
        return orer(l1, noter(l2))
    elif(op == 'and not'):
        return ander(l1, noter(l2))
    else:
        print('Invalid operation')
        return -1, []

def process(query, ops):
    tokens = preprocess([query])
    ops = [op.lower().strip() for op in ops]
    print('Query tokens:', tokens)
    print('Operations:', ops)
    comparisons = 0
    
    # Input is assumed to be in correct format
    if(tokens and ops):
        l1 = index[tokens[0]][1]
        for i in range(len(ops)):
            l2 = index[tokens[i+1]][1]
            numc, l1 = handle(l1, l2, ops[i])
            if(numc == -1):
                return
            comparisons += numc
        
        return comparisons, l1
    else:
        print('Empty input')

## D: Testing

In [126]:
query = 'lion stood thoughtfully for a moment'
ops = ['or', 'or', 'or']
ops1 = ['or', 'and', 'or not']
ops2 = ['or', 'or', 'and not']
ops3 = ['or', 'and', 'or not']
ops4 = ['or', 'or', 'or not']

In [127]:
a = process(query, ops)
print(a[0], len(a[1]))

Query tokens: ['lion', 'stood', 'thought', 'moment']
Operations: ['or', 'or', 'or']
860 408


In [128]:
a = process(query, ops1)
print(a[0], len(a[1]))

Query tokens: ['lion', 'stood', 'thought', 'moment']
Operations: ['or', 'and', 'or not']
1455 1013


In [129]:
a = process(query, ops2)
print(a[0], len(a[1]))

Query tokens: ['lion', 'stood', 'thought', 'moment']
Operations: ['or', 'or', 'and not']
1530 253


In [130]:
a = process(query, ops3)
print(a[0], len(a[1]))

Query tokens: ['lion', 'stood', 'thought', 'moment']
Operations: ['or', 'and', 'or not']
1455 1013


In [131]:
a = process(query, ops4)
print(a[0], len(a[1]))

Query tokens: ['lion', 'stood', 'thought', 'moment']
Operations: ['or', 'or', 'or not']
1530 1081


In [132]:
input_query = input('Enter the query: ')
print('\nEnter operations separated by ",",')
input_ops = list(map(str,input("\nEnter the operations : ").strip().split(',')))

Enter the query: telephone paved roads

Enter operations separated by ",",

Enter the operations : or not, and not


In [136]:
result = process(input_query, input_ops)
print('\nNumber of documents matched:', len(result[1]))
print('Number of comparisons:', result[0])
print('\nDocument list\n')

for docid in result[1]:
    print(docnames[docid])

Query tokens: ['telephon', 'pave', 'road']
Operations: ['or not', 'and not']

Number of documents matched: 999
Number of comparisons: 2239

Document list

1st_aid.txt
abbott.txt
acetab1.txt
acne1.txt
acronym.txt
adameve.hum
adcopy.hum
addrmeri.txt
admin.txt
adrian_e.faq
ads.txt
adt_miam.txt
advrtize.txt
aeonint.txt
age.txt
aggie.txt
airlines
alabama.txt
alcatax.txt
alcohol.hum
alflog.txt
allusion
all_grai
ambrose.bie
amchap2.txt
analogy.hum
aniherb.txt
anime.lif
anim_lif.txt
annoy.fascist
anorexia.txt
answers
anthropo.stu
antibiot.txt
antimead.bev
aphrodis.txt
appbred.brd
appetiz.rcp
applepie.des
apsaucke.des
apsnet.txt
arab.dic
arcadian.txt
argotdic.txt
arnold.txt
art-fart.hum
arthriti.txt
atherosc.txt
atombomb.hum
att.txt
aussie.lng
avengers.lis
awespinh.sal
ayurved.txt
a_fish_c.apo
a_tv_t-p.com
b-2.jok
b12.txt
back1.txt
bad
bad-d
bad.jok
badday.hum
bagelope.txt
bakebred.txt
baklava.des
banana01.brd
banana02.brd
banana03.brd
banana04.brd
banana05.brd
bank.rob
barney.cn1
basehead.txt
